<a href="https://colab.research.google.com/github/baharahmadpoor/Hogwarts-Portal/blob/main/ODE_PINN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ODE:
 du/dx = cos(x), x in [0, 1]

with initial condition:
 u(0) = 0

with exact solution:
 u(x) = sin(x)

In [26]:
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

In [27]:
def f_rhs(x):
  return torch.cos(x)

def u_exact(x):
  return torch.sin(x)

a,b = 0,1

In [28]:
class PINN(nn.Module):
  def __init__(self):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(1, 32),
        nn.Tanh(), # activation function
        nn.Linear(32, 32),
        nn.Tanh(), # activation function
        nn.Linear(32, 1)
    )

  def forward(self, x):
        return self.net(x)

model = PINN()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [29]:
# training points (collocation)
x_train = torch.linspace(a, b, 40).view(-1, 1) # (b-a)/40
x_train.requires_grad_(True)

tensor([[0.0000],
        [0.0256],
        [0.0513],
        [0.0769],
        [0.1026],
        [0.1282],
        [0.1538],
        [0.1795],
        [0.2051],
        [0.2308],
        [0.2564],
        [0.2821],
        [0.3077],
        [0.3333],
        [0.3590],
        [0.3846],
        [0.4103],
        [0.4359],
        [0.4615],
        [0.4872],
        [0.5128],
        [0.5385],
        [0.5641],
        [0.5897],
        [0.6154],
        [0.6410],
        [0.6667],
        [0.6923],
        [0.7179],
        [0.7436],
        [0.7692],
        [0.7949],
        [0.8205],
        [0.8462],
        [0.8718],
        [0.8974],
        [0.9231],
        [0.9487],
        [0.9744],
        [1.0000]], requires_grad=True)

In [30]:
Epochs = 3000

for epoch in range(Epochs):
  optimizer.zero_grad()

  # network output
  u = model(x_train)

  # du/dx via autogard
  du_dx = torch.autograd.grad(
      u,
      x_train,
      grad_outputs=torch.ones_like(u),
      create_graph=True
  )[0]

  # ODE residual
  residual =  du_dx - f_rhs(x_train)

  # initial condition u(0) = 0
  u0 = model(torch.tensor([0.0]))

  # Total loss = residual loss + initial loss
  loss = torch.mean(residual**2) + (u0 - 0.0)**2

  loss.backward()
  optimizer.step()

  if epoch % 500 == 0:
      print(f"Epoch {epoch}, Loss = {loss.item():.3e}")

Epoch 0, Loss = 4.009e-01
Epoch 500, Loss = 9.746e-05
Epoch 1000, Loss = 6.067e-05
Epoch 1500, Loss = 4.804e-05
Epoch 2000, Loss = 3.339e-05
Epoch 2500, Loss = 1.754e-05


In [31]:
x_test = torch.linspace(a, b, 200).view(-1, 1) # (b-a)/200

In [32]:
u_pred = model(x_test).detach().numpy()
u_true = u_exact(x_test).numpy()

In [33]:
# mean absolut error
mae = np.mean(np.abs(u_pred - u_true))
print(f"Test Mean: {mae}")

Test Mean: 0.00019516250176820904


In [34]:
# mean square error
mse = np.mean(np.square(u_pred - u_true))
print(f"Test MSE: {mse}")

Test MSE: 4.8634465343866395e-08


In [35]:
rmse = np.sqrt(np.mean(np.square(u_pred - u_true)))
print(f"Test RMSE: {rmse}")

Test RMSE: 0.00022053223801776767


k.parand1394@gmail.com


y"(x) + (2/x)y'(x) + y(x) = 0

y(0) = 1, y'(0) = 0

n = 0 -> u(x) = 1-x^2/6

n = 1 -> u(x) = sin x/x

n = 5 -> u(x) = 1/radical(1+x^2/3)

In [36]:
def exact_solution(x, n):

    x_np = x.detach().numpy() if torch.is_tensor(x) else x

    if n == 0:
        return 1 - x_np**2/6
    elif n == 1:
        result = np.ones_like(x_np)
        mask = x_np != 0
        result[mask] = np.sin(x_np[mask]) / x_np[mask]
        return result
    elif n == 5:
        return 1 / np.sqrt(1 + x_np**2/3)

In [37]:
class PINN_Bessel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(1, 32),
            nn.Tanh(),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Linear(32, 32),
            nn.Tanh(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)

# n = 1
model = PINN_Bessel()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [38]:
a, b = 0, 10  # interval
N = 50  # Number of points

# Chebyshev roots of the first kind
k = np.arange(1, N + 1)
cheb_points = np.cos(np.pi * (2*k - 1) / (2*N))

x_numpy = a + (b - a) * (cheb_points + 1) / 2

x_train = torch.tensor(x_numpy, dtype=torch.float32).view(-1, 1)
x_train.requires_grad_(True)

tensor([[9.9975e+00],
        [9.9778e+00],
        [9.9384e+00],
        [9.8796e+00],
        [9.8015e+00],
        [9.7044e+00],
        [9.5888e+00],
        [9.4550e+00],
        [9.3037e+00],
        [9.1354e+00],
        [8.9508e+00],
        [8.7506e+00],
        [8.5355e+00],
        [8.3066e+00],
        [8.0645e+00],
        [7.8104e+00],
        [7.5452e+00],
        [7.2700e+00],
        [6.9857e+00],
        [6.6937e+00],
        [6.3950e+00],
        [6.0907e+00],
        [5.7822e+00],
        [5.4705e+00],
        [5.1571e+00],
        [4.8429e+00],
        [4.5295e+00],
        [4.2178e+00],
        [3.9093e+00],
        [3.6050e+00],
        [3.3063e+00],
        [3.0143e+00],
        [2.7300e+00],
        [2.4548e+00],
        [2.1896e+00],
        [1.9355e+00],
        [1.6934e+00],
        [1.4645e+00],
        [1.2494e+00],
        [1.0492e+00],
        [8.6460e-01],
        [6.9629e-01],
        [5.4497e-01],
        [4.1123e-01],
        [2.9560e-01],
        [1

In [39]:
x0 = torch.tensor([[1e-3]], dtype=torch.float32, requires_grad=True)

In [40]:
epochs = 5000
loss_history = []

for epoch in range(epochs):
    optimizer.zero_grad()
    y = model(x_train)

    dy_dx = torch.autograd.grad(
        y, x_train,
        grad_outputs=torch.ones_like(y),
        create_graph=True
    )[0]

    d2y_dx2 = torch.autograd.grad(
        dy_dx, x_train,
        grad_outputs=torch.ones_like(dy_dx),
        create_graph=True
    )[0]

    eps = 1e-8
    residual = d2y_dx2 + (2/(x_train + eps)) * dy_dx + y
    loss_eq = torch.mean(residual**2)


    y0 = model(x0)
    loss_ic1 = (y0 - 1.0)**2  # y(0) = 1

    dy0_dx = torch.autograd.grad(
        y0, x0,
        grad_outputs=torch.ones_like(y0),
        create_graph=True
    )[0]
    loss_ic2 = (dy0_dx - 0.0)**2  # y'(0) = 0

    loss = loss_eq + 10.0 * loss_ic1 + 10.0 * loss_ic2

    loss.backward()
    optimizer.step()
    loss_history.append(loss.item())

    if epoch % 1000 == 0:
        print(f"Epoch {epoch:4d}, Loss = {loss.item():.3e}")

Epoch    0, Loss = 7.680e+01
Epoch 1000, Loss = 1.534e-02
Epoch 2000, Loss = 1.287e-02
Epoch 3000, Loss = 1.167e-02
Epoch 4000, Loss = 1.066e-02


In [41]:
x_test_np = np.linspace(a, b, 200)
x_test = torch.tensor(x_test_np, dtype=torch.float32).view(-1, 1)

with torch.no_grad():
    y_pred = model(x_test).numpy()

y_exact = exact_solution(x_test_np, n)

mae = np.mean(np.abs(y_pred.flatten() - y_exact))
mse = np.mean((y_pred.flatten() - y_exact)**2)
rmse = np.sqrt(mse)

print(f"MAE  = {mae:.2e}")
print(f"MSE  = {mse:.2e}")
print(f"RMSE = {rmse:.2e}")

MAE  = 9.44e-02
MSE  = 1.46e-02
RMSE = 1.21e-01


In [47]:
def train_for_n(n=1):

    model = PINN_Bessel()
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

    N = 50
    k = np.arange(1, N+1)
    cheb = np.cos(np.pi * (2*k - 1) / (2*N))
    x_train_np = 10 * (cheb + 1) / 2

    x_train = torch.tensor(x_train_np, dtype=torch.float32).view(-1, 1)
    x_train.requires_grad_(True)

    for epoch in range(5000):
        optimizer.zero_grad()

        y = model(x_train)
        dy = torch.autograd.grad(y, x_train, torch.ones_like(y), create_graph=True)[0]
        d2y = torch.autograd.grad(dy, x_train, torch.ones_like(dy), create_graph=True)[0]

        residual = d2y + (2/(x_train + 1e-8))*dy + y
        loss_eq = torch.mean(residual**2)

        x0 = torch.tensor([[0.001]], dtype=torch.float32, requires_grad=True)
        y0 = model(x0)
        dy0 = torch.autograd.grad(y0, x0, torch.ones_like(y0), create_graph=True)[0]

        loss_ic = (y0 - 1)**2 + (dy0 - 0)**2
        loss = loss_eq + 10*loss_ic

        loss.backward()
        optimizer.step()

        if epoch % 1000 == 0:
            print(f"{epoch}: loss = {loss.item():.3e}")

    x_test = torch.linspace(0, 10, 200).view(-1, 1)
    with torch.no_grad():
        y_pred = model(x_test).numpy().flatten()

    y_true = exact_solution(x_test.numpy().flatten(), n)

    mae = np.mean(np.abs(y_pred - y_true))
    mse = np.mean((y_pred - y_true)**2)
    rmse = np.sqrt(mse)

    print(f"MAE  = {mae:.2e}")
    print(f"MSE  = {mse:.2e}")
    print(f"RMSE = {rmse:.2e}")

    return mae, mse, rmse

In [48]:
errors = {}
for n_value in [0, 1, 5]:
    mae, mse, rmse = train_for_n(n_value)
    errors[n_value] = (mae, mse, rmse)

print("\n" + "="*60)
print("="*60)
print("n |      MAE      |      MSE      |     RMSE")
print("-"*60)

for n_val in [0, 1, 5]:
    mae, mse, rmse = errors[n_val]
    print(f"{n_val} | {mae:12.2e} | {mse:12.2e} | {rmse:12.2e}")

0: loss = 1.172e+01
1000: loss = 1.269e-02
2000: loss = 1.187e-02
3000: loss = 9.417e-03
4000: loss = 9.365e-03
MAE  = 4.74e+00
MSE  = 4.54e+01
RMSE = 6.74e+00
0: loss = 6.406e+00
1000: loss = 1.772e-02
2000: loss = 1.304e-02
3000: loss = 1.113e-02
4000: loss = 9.611e-03
MAE  = 9.60e-02
MSE  = 1.48e-02
RMSE = 1.22e-01
0: loss = 4.622e+02
1000: loss = 3.753e-02
2000: loss = 1.672e-02
3000: loss = 1.353e-02
4000: loss = 1.143e-02
MAE  = 2.44e-01
MSE  = 7.04e-02
RMSE = 2.65e-01

n |      MAE      |      MSE      |     RMSE
------------------------------------------------------------
0 |     4.74e+00 |     4.54e+01 |     6.74e+00
1 |     9.60e-02 |     1.48e-02 |     1.22e-01
5 |     2.44e-01 |     7.04e-02 |     2.65e-01


n = 0 -> Errors are high (MAE ≈ 4.74) → model failed to learn well. To improve, we can use more epochs and fewer neurons.

n = 1 -> Very good! (MAE ≈ 0.096) → model has almost learned the correct answer

n = 5 -> Acceptable (MAE ≈ 0.244) Better than n=0, worse than n=1